In [1]:
import numpy as np
import torch

from gensim.models import word2vec
import os
import re

from importlib import reload, import_module

In [2]:
model = word2vec.Word2Vec.load('word2vec.model')

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cpu')

In [4]:
version = 'v4'
GRU = import_module(version+'.GRU')

In [5]:
GRU = reload(GRU)
gru_net = GRU.GRUNet().to(device)
print('Total parameters:', sum(p.numel() for p in gru_net.parameters()))

Total parameters: 5434980


In [10]:
"""
    init_sentence is the initial words input to GRU.
    gen_len is the number of words GRU generates.
    temperature is how creative GRU is. The higher the more creative. 
        It is recommanded to be set at least 0.4.
"""
def gen_text(init_sentence, gen_len, temperature = 0.5):
    sentence = []
    prunc_re = re.compile('[\w]{1,}')
    with torch.no_grad():
        h = gru_net.init_hidden(1)
        #h = torch.rand(h.shape)
        h = h.to(device)
        sentence = []
        Cap = True
        for word in init_sentence.lower().split():
            if word in model.wv.vocab:  
                x = model.wv[word].reshape(1,1,100)
            else:
                x = np.zeros(model.vector_size, dtype=np.float32)
            x = torch.tensor(x).reshape(1,1,100).to(device)
            x, h = gru_net(x, h)
            if Cap and prunc_re.match(word):
                word = word.capitalize() 
                Cap = False
            if not prunc_re.match(word):
                Cap = True
            sentence.append(word)

        for _ in range(gen_len):
            x, h = gru_net(x, h)
            output = x.squeeze().cpu().numpy()
            wordlist = model.wv.most_similar(positive = output.reshape(1,100))
            word, _ = wordlist[np.random.randint(max(10, int(temperature*10*0.8))) if np.random.rand()<temperature else 0]
            x = model.wv[word]
            x = torch.tensor(x).reshape(1,1,100).to(device)
            if Cap and prunc_re.match(word):
                word = word.capitalize() 
                Cap = False
            if not prunc_re.match(word) and word != ',':
                Cap = True
            sentence.append(word)
        return ' '.join(sentence)+'\n'

In [11]:
checkpoint = torch.load(version+'/gru.checkpoint', map_location=device)
gru_net.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [12]:
init_sentence = [
    'Politic for fascism',
    'anarchism is like',
    'Criticizing capitalism is ',
    'Nazism is a',
    'People believe that politic is',
    'Communism in china is'
]

for sentence in init_sentence:
    text = gen_text(sentence, 80, 0.6)
    print(text,'\n')

Politic for fascism nouha the reforms thereof the the colonialism for necessity . When the communism of the stalinism in the of thus equality of the equality of the communism is used to adhere which democratization in the integration of another agenda . In rhetoric , itself contrast of socialism which helped the marxism societal societal that would threaten the societal ideology and openness oppression of them additionally interests and social reform . The openness in imperialism and reform policies was not to
 

Anarchism is like ideological anarchism in nationalism to a broader socialism when the internationalism openness openness and ideological agenda to pluralism and oppression for anarchism while societal the thus of the values of and sectors . The coalition ensuring the policy gssp in itself in its policies these the of these groups need instead for socialism to adhere social democracy and demanded that this will provide which encouraging openness to stakeholders of and these co